<a href="https://colab.research.google.com/github/fabilj825/EntityLinkerUmls/blob/main/spacy_knowledge_base.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# `spacy_knowledge_base`

---


Diese Zelle muss ausgeführt werden, immer wieder wenn man eine neue Session startet.

Die Ausführung verknüpft den Code mit dem Google Drive, sodaß die Änderungen gespeichert können werden.

In [1]:
from google.colab import drive
drive.mount('/content/drive')

gdrive_base_address = "drive/MyDrive/UMLS/"
dataframes_address = gdrive_base_address + "dataframes/"
knowledgebase_address = gdrive_base_address + "knowledgebase/"

Mounted at /content/drive


Diese Zelle muss ausschließlich bei der erste Ausführung, benutzt werden, um ein Virtual Environment zu aktivieren und die Datenbank zu installieren.

In [2]:
!pip install spacy[cuda12x]
!pip install kaleido
!pip install python-multipart
!pip install uvicorn
!pip install fastapi
!pip install wandb
!pip install scispacy
!pip install spacy-transformers
!pip install https://s3-us-west-2.amazonaws.com/ai2-s2-scispacy/releases/v0.5.3/en_core_sci_lg-0.5.3.tar.gz
!pip install https://s3-us-west-2.amazonaws.com/ai2-s2-scispacy/releases/v0.5.3/en_ner_bc5cdr_md-0.5.3.tar.gz
!pip install https://s3-us-west-2.amazonaws.com/ai2-s2-scispacy/releases/v0.5.3/en_core_sci_scibert-0.5.3.tar.gz

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 11.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 468.9/468.9 kB 17.8 MB/s eta 0:00:00
created virtual environment CPython3.10.12.final.0-64 in 16572ms
  creator CPython3Posix(dest=/content/drive/MyDrive/colab_env, clear=False, no_vcs_ignore=False, global=False)
  seeder FromAppData(download=False, pip=bundle, setuptools=bundle, wheel=bundle, via=copy, app_data_dir=/root/.local/share/virtualenv)
    added seed packages: pip==23.3.1, setuptools==69.0.2, wheel==0.42.0
  activators BashActivator,CShellActivator,FishActivator,NushellActivator,PowerShellActivator,PythonActivator
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 531.2/531.2 MB 2.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for en-core-sci-lg: filename=en_core_sci_lg-0.5.3-py3-none-any.whl size=531476666 sha256=86634ac339bc14353dfd17d4bdc54f17e373ade59d856ed0bfd7297c690684e6
  Stored in directory: /root/.cache/pip/wheels/98/4

## ***Anfang des Python Codes***

In [4]:
import spacy
import os
from spacy.kb import InMemoryLookupKB
import csv
from pathlib import Path
from collections import defaultdict

ENTITY_VEC_LEN = 200

Lade die Datenbank

In [5]:
nlp = spacy.load("en_core_sci_lg")

/usr/local/lib/python3.10/dist-packages/spacy/language.py:2141: FutureWarning: Possible set union at position 6328
  deserializers["tokenizer"] = lambda p: self.tokenizer.from_disk(  # type: ignore[union-attr]


In [6]:
def load_entities(path):
    entities_loc = path
    names = dict()
    defintions = dict()
    with entities_loc.open("r", encoding="utf8") as csvfile:
        csvreader = csv.reader(csvfile, delimiter=",")
        for row in csvreader:
            cui= row[1]
            desc = row[2]
            if cui in defintions.keys():
                continue
            defintions[cui] = desc
    return defintions

In [7]:
def_dict = load_entities(Path.cwd() / dataframes_address / "MRDEF.csv")

In [8]:
kb = InMemoryLookupKB(vocab=nlp.vocab, entity_vector_length=ENTITY_VEC_LEN)

In [10]:
for cui, desc in def_dict.items():
    desc_doc = nlp(desc)
    desc_enc = desc_doc.vector
    kb.add_entity(entity=cui, entity_vector=desc_enc, freq=342)   # 342 is an arbitrary value here

Streaming output truncated to the last 5000 lines.
<ipython-input-10-ccbba288ee09>:4: UserWarning: [W018] Entity 'C5668575' already exists in the Knowledge Base - ignoring the duplicate entry.
  kb.add_entity(entity=cui, entity_vector=desc_enc, freq=342)   # 342 is an arbitrary value here
<ipython-input-10-ccbba288ee09>:4: UserWarning: [W018] Entity 'C5668576' already exists in the Knowledge Base - ignoring the duplicate entry.
  kb.add_entity(entity=cui, entity_vector=desc_enc, freq=342)   # 342 is an arbitrary value here
<ipython-input-10-ccbba288ee09>:4: UserWarning: [W018] Entity 'C5668577' already exists in the Knowledge Base - ignoring the duplicate entry.
  kb.add_entity(entity=cui, entity_vector=desc_enc, freq=342)   # 342 is an arbitrary value here
<ipython-input-10-ccbba288ee09>:4: UserWarning: [W018] Entity 'C5668578' already exists in the Knowledge Base - ignoring the duplicate entry.
  kb.add_entity(entity=cui, entity_vector=desc_enc, freq=342)   # 342 is an arbitrary valu

In [11]:
def load_entities_conso(path):
    entities_loc = path
    cuis = defaultdict(list)
    with entities_loc.open("r", encoding="utf8") as csvfile:
        csvreader = csv.reader(csvfile, delimiter=",")
        for row in csvreader:
            cui= row[1]
            desc = row[2]
            cuis[cui].append(desc)
    return cuis

In [12]:
des_dict = load_entities_conso(Path.cwd() / dataframes_address / "MRCONSO.csv")

In [13]:
for cui, desc in des_dict.items():
    num_desc=len(desc)
    prob=1/num_desc
    for des in desc:
        kb.add_alias(alias=des, entities=[cui], probabilities=[prob])

Streaming output truncated to the last 5000 lines.
<ipython-input-13-72dfc869baeb>:5: UserWarning: [W017] Alias 'Anti-p200 pemphigoid' already exists in the Knowledge Base.
  kb.add_alias(alias=des, entities=[cui], probabilities=[prob])
<ipython-input-13-72dfc869baeb>:5: UserWarning: [W017] Alias 'Familial congenital nasolacrimal duct obstruction' already exists in the Knowledge Base.
  kb.add_alias(alias=des, entities=[cui], probabilities=[prob])
<ipython-input-13-72dfc869baeb>:5: UserWarning: [W017] Alias 'Polyclonal hyperviscosity syndrome' already exists in the Knowledge Base.
  kb.add_alias(alias=des, entities=[cui], probabilities=[prob])
<ipython-input-13-72dfc869baeb>:5: UserWarning: [W017] Alias 'Isolated focal non-epidermolytic palmoplantar keratoderma' already exists in the Knowledge Base.
  kb.add_alias(alias=des, entities=[cui], probabilities=[prob])
<ipython-input-13-72dfc869baeb>:5: UserWarning: [W017] Alias 'Supratip dysplasia' already exists in the Knowledge Base.
  kb.

In [14]:
print(f"Entities in the KB: {kb.get_entity_strings()}")
print(f"Aliases in the KB: {kb.get_alias_strings()}")

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [15]:
os.makedirs(knowledgebase_address, exist_ok=True)

In [16]:
kb.to_disk(knowledgebase_address + 'mykb')

In [17]:
kb = InMemoryLookupKB(vocab=nlp.vocab, entity_vector_length=ENTITY_VEC_LEN)
kb.from_disk(knowledgebase_address + 'mykb')

In [18]:
print(f"Entities in the KB: {len(kb.get_entity_strings())}")
print(f"Aliases in the KB: {len(kb.get_alias_strings())}")

Entities in the KB: 264500
Aliases in the KB: 2619149
